In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
data_path = Path.cwd().joinpath("datasets", "halifax_site", "output.json")

In [ ]:
data_path.exists()

In [ ]:
### Read file from json in pandas dataframe
halifax_text_df  = pd.read_json(data_path)

In [ ]:
halifax_text_df["content"] = halifax_text_df.content.str.replace(
    r'\n{1,}', ' ', regex=True)

In [ ]:
halifax_text_df.content.drop_duplicates(keep="first")

In [ ]:
halifax_text_df = halifax_text_df[~halifax_text_df.content.duplicated()]

In [ ]:
halifax_text_df.head(10)

In [ ]:
halifax_text_df.title.loc[halifax_text_df.title.str.endswith("?")].drop_duplicates(keep="first")

We don't have a huge dataset, most of our dataset is duplicated. However we have around 1686 data point we can work with! Most of those point may or may not be related to mortagage, but they will do the work.

In [ ]:
halifax_text_df.head()

In [ ]:
halifax_text_df

In [ ]:
test_dataset = halifax_text_df.loc[halifax_text_df.title.loc[halifax_text_df.title.str.endswith(
    "?")].duplicated().index]

In [ ]:
test_dataset = test_dataset.reset_index()

In [ ]:
test_dataset["content_cleaned"] = test_dataset.apply(lambda x: x.content.replace(x.title, '').strip(), axis="columns")

In [ ]:
test_dataset.iloc[1].content

In [ ]:
test_dataset.iloc[1].content

In [ ]:
test_dataset.iloc[1].content_cleaned

In [ ]:
test_dataset

In [ ]:
train_dataset = halifax_text_df.loc[~halifax_text_df.index.isin(test_dataset.index)]

In [ ]:
train_dataset.head()

Now have our train and test dataset, we need to move to the next step step which involve, building the pipeline which will index the documeent in our vector database and then respond to the question.

train_datatset.shape

In [ ]:
train_dataset.shape

In [ ]:
train_dataset.sample(10)

In [ ]:
train_dataset.head()

In [ ]:
train_dataset["content_cleaned"] = train_dataset.apply(lambda x: x.content.replace(x.title.strip(), '').strip(), axis="columns")
test_dataset["content_cleaned"] = test_dataset.apply(lambda x: x.content.replace(x.title.strip(), '').strip(), axis="columns")

In [ ]:
train_dataset = train_dataset.loc[train_dataset.content_cleaned.str.len() != 0]

In [ ]:
train_dataset.head()

Next we will be using the model  to encode the characters.



bitext/Mistral-7B-Mortgage-Loans

We will be using that model to learn the embeddings.

https://github.com/AI4Finance-Foundation/FinGPT?tab=readme-ov-file

GPT like models cannot generalize embedding well. need to find a good model to use to learn embedding.

dunzhang/stella_en_1.5B_v5

This will be best language model to use, it is small and match the quality.

Need to learn and find out what is the shape fo the models.


response

### Experiment with support for other models.

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model_path = Path.cwd().joinpath("models")

In [ ]:
model_id = "dunzhang/stella_en_400M_v5"

In [ ]:
model = SentenceTransformer(
    model_path.joinpath(model_id).__str__(),
    trust_remote_code=True,
    device="cpu",
    config_kwargs={"use_memory_efficient_attention": False, "unpad_inputs": False},
    local_files_only=True
)

In [ ]:
query_prompt_name = "s2p_query"
queries = [
    "What are some ways to reduce stress?",
    "What are the benefits of drinking green tea?",
    "How often do mortgage rates change?",
]
# docs do not need any prompts
docs = [
    "There are many effective ways to reduce stress. Some common techniques include deep breathing, meditation, and physical activity. Engaging in hobbies, spending time in nature, and connecting with loved ones can also help alleviate stress. Additionally, setting boundaries, practicing self-care, and learning to say no can prevent stress from building up.",
    "Green tea has been consumed for centuries and is known for its potential health benefits. It contains antioxidants that may help protect the body against damage caused by free radicals. Regular consumption of green tea has been associated with improved heart health, enhanced cognitive function, and a reduced risk of certain types of cancer. The polyphenols in green tea may also have anti-inflammatory and weight loss properties.",
    " Mortgage rates can stay the same for years or they can change often, depending on various factors and market conditions. Factors like the Bank of England Base Rate, the housing market and the state of the economy can affect mortgage rates. But if you have a fixed rate mortgage, your rate stays the same for the length of your deal.  If you’ve already applied for a Halifax mortgage, the rate we’ve offered you won’t be affected by any changes to our mortgage range once the mortgage illustration has been accepted. "
]

In [ ]:
query_embeddings = model.encode(queries, prompt_name=query_prompt_name)
doc_embeddings = model.encode(docs)

In [ ]:
similarities = model.similarity(query_embeddings, doc_embeddings)
print(similarities)

In [ ]:
train_dataset.head()

In [ ]:
train_dataset.head()

In [ ]:
test_dataset.head()

In [ ]:
test_dataset

In [ ]:
sample = test_dataset.sample(10)

In [ ]:
sample_queries = sample["title"].tolist()
sample_docs = sample["content_cleaned"].tolist()

In [ ]:
sample

In [ ]:
sample_queries_embeddings = model.encode(sample_queries, prompt_name=query_prompt_name)
sample_docs_embeddings = model.encode(sample_docs)

similarities = model.similarity(sample_queries_embeddings, sample_docs_embeddings)

In [ ]:
import seaborn as sb

In [ ]:
sb.heatmap(similarities, cmap="Blues", annot=True)

In [ ]:
sample.iloc[4]

We need to do more cleaning.

We can already ingest this document and train our rag on it.

The next stip will be to ingest the model in the database. and read from them.

training_set

In [ ]:
train_dataset.head()

In [ ]:
train_dataset = train_dataset[~train_dataset.content.isna()
                              & ~train_dataset.url.isna()]
test_dataset = test_dataset[~test_dataset.content.isna() & ~test_dataset.url.isna()]

In [ ]:
dataset_path = Path.cwd().joinpath("datasets", "halifax_site")

In [ ]:
train_dataset.to_csv(dataset_path.joinpath("training_data.csv"), sep="|", index=False)

In [ ]:
test_dataset.to_csv(dataset_path.joinpath("test_data.csv"), sep="|", index=False)

In [ ]:
train_dataset.head()

In [ ]:
train_dataset.shape

tra